# N-body Simulations and More

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# NOTE: All units in SI
from scipy.constants import gravitational_constant, au

year = 365.25*24*3600 # seconds

mass_sun = 1.989e30 # kg
mass_earth = 5.972e24 # kg
mass_mars = 6.39e23 # kg
mass_jupiter = 1.898e27 # kg

earth_distance = 1.496e11 # meters
mars_distance = 227.9*1.e9 # meters
jupiter_distance = 778.57*1.e9 # meters

scale_factor = (gravitational_constant*mass_sun*year**2/earth_distance**3)

# Masses and distances in our nicely scaled units
mE = mass_earth/mass_sun
mM = mass_mars/mass_sun
mJ = mass_jupiter/mass_sun

rE = earth_distance / earth_distance
rM = mars_distance / earth_distance
rJ = jupiter_distance / earth_distance

## A Realistic Star System

Let's now simulate a realistic solar system. We'll include the Sun and a few planets, and they will all have mass and interact.

We'll make this entirely general, so it can handle N different objects in 3D. Each object will have six variables associated with it - $(x, y, z, v_x, v_y, v_z)$.

Because of the way `odeint` works, we need to store all of the variables in a single 1D array that we'll call `state`. We'll store them in this order in `state`:
 * positions for object `i`: `6*i` to `6*i+3`
 * velocities for object `i`: `6*i+3` to `6*i+6`

Writing these out every time they're needed would make the code very hard to read and debug. So we'll define some helper functions and a class to help process the data. This is the main point of the notebook - it's not so much about solving the ODEs, it's about being able to process the output.

Let's start by defining our derivatives function.

In [2]:
def distance(pos1, pos2):
    return np.sqrt(np.sum((pos2-pos1)**2, axis=-1))

def deriv(state, t):
    derivatives = 0.*state # make an empty array shaped like state
    for i in range(len(mass)):
        pos1 = state[6*i: 6*i+3]
        acc = 0.
        for j in range(len(mass)):
            if j == i: # Object doesn't interact with itself
                continue

            pos2 = state[6*j: 6*j+3]

            r = distance(pos1, pos2)
            vec = pos2 - pos1
            acc += scale_factor*mass[j]*vec/r**3

        # Derivative of position is velocity
        derivatives[6*i: 6*i+3] = state[6*i+3: 6*i+6]
        # Derivative of velocity is acceleration
        derivatives[6*i+3: 6*i+6] = acc

    return derivatives


## Exercise

Set up initial conditions for a solar system with the Sun, the Earth, and Jupiter. Remember that the velocity should be $2 \pi / \sqrt{r}$ in our nicely scaled units. Solve this for 100 years. Then plot the $x, y$ positions of the three objects over time. What is wrong with the solution?

In [3]:
mass = np.array([1., mE, mJ])

In [4]:
times = # ??

ic = np.array([])
soln = odeint #??

SyntaxError: invalid syntax (3052957143.py, line 1)

In [ ]:
# Make the plot here

## Exercise

The centre of mass is not at rest by default (the total momentum is not zero). Fill in the functions below to calculate position and velocity of each object, then calculate the total momentum

$$p_\mathrm{total} = \sum\limits_{i} ~ m_i \vec{v_i}$$

Divide this by the total mass to get the velocity of the center of mass.

In [ ]:
def pos(state, i):
    """The position (3D) of object i"""
    return # ??

def vel(state, i):
    """The velocity (3D) of object i"""
    return # ??

def total_momentum(state, mass):
    p_sum = 0.
    for i in range(len(mass)):
        p_sum +=  # ??
    return p_sum

## Exercise

Now fill in the function to remove the center of mass motion, make and plot the new solution.

In [ ]:
def remove_center_of_mass_motion(state):
    new_state = state.copy() # Don't modify the state, create a new one
    
    # ??

    return new_state


In [ ]:
new_ic = remove_center_of_mass_motion(ic)
new_ic

In [ ]:
# Make the plot here

## A class to simplify the analysis

Now we want to investigate things like energy conservation. We'll build a class to store the result of the simulation, and add functions to it to calculate the total momentum, total kinetic energy, and total potential energy.

$$\mathrm{KE} = \sum\limits_{i} \frac{1}{2} m_i |\vec{v_i}|^2$$

$$\mathrm{PE} = - \sum\limits_{i<j} \frac{G m_i m_j}{r_{i,j}}$$

The gravitational constant $G$ can be replaced by `scale_factor` in the units we use in the code.

When we write the code for these, we need to sum the $|\vec{v}_i|^2$ over the three dimensions. But there is also a time dimension and we don't want to sum over that. We can do this using the `axis` option in `np.sum`. If we do `np.sum(arr, axis=-1)`, it will only sum over the last dimension of the array, and leave the time dimension alone. It's helpful to check the shapes of the inputs and outputs as you write the functions below.

### Exercise

Fill in the class below with functions for total momentum, total kinetic, and total potential energies. Look at the derivatives function for inspiration. This is difficult, so take your time and test your output.

In [ ]:
class NBody():
    def __init__(self, mass, soln):
        self.n_obj = len(mass)
        self.mass = mass
        self.soln = soln
    @property
    def m_total(self):
        return np.sum(mass)
    def pos(self, i):
        return self.soln[:, 6*i:6*i+3]
    def vel(self, i):
        return self.soln[:, 6*i+3:6*i+6]
    def momentum(self, i):
        # ??
    
    def total_kinetic(self):
        # ??
    
    def total_potential(self):
        # ??


### Exercise

Now load your simulation into the `NBody` class, calculate the total energy, and show that it is conserved. Is the conservation perfect? Does it oscillate?

In [ ]:
nb = NBody(mass, soln)

In [ ]:
Ktot = nb.total_kinetic()
Vtot = nb.total_potential()

### Exercise

Now see how long you can run the simulation for. How well is the energy conserved? This is an indication of how accurate the simulation is. Numerical errors will cause the total energy to change.

### Exercise

Try decreasing the accuracy of the solver, using the options `rtol` and `atol`. Can you make the simulation run faster by decreasing these? Does using fewer output points in the `times` series make a difference? What's the longest time you can simulate while keeping reasonable accuracy of the total energy?

## Binary Star Systems

### Exercise

Now simulate a binary star system. Start with equal-mass stars. You'll need to change the `mass` array. It will be easy to get an elliptical orbit. You'll be able to see that it's elliptical because the kinetic and potential energies will oscillate (though the sum won't).

### Exercise

Choose the initial velocities that make the orbits circular (meaning that the distance doesn't change over time, or the KE and PE don't oscillate). You can do this by numerically solving for a solution where the distance doesn't vary, or do the calculation analytically. Either way, check your solution.

### Exercise

Now create and test a binary with unequal masses. Start with a two-to-one ratio, but can you write a function to create the initial conditions for any masses, and any separation? Test that you have circular motion.

### Exercise

Now try replacing the Sun with two stars of $0.5 M_\odot$, in an orbit much smaller than 1 AU. Is the solar system still stable?

### Exercise

If you have extra time, try experimenting with trinary systems. Put two stars at the center, and one orbiting them both at a larger distance. What kind of solutions can you get?

We will not provide a solution to this exercise